In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [2]:
response = requests.get('https://data.cityofnewyork.us/resource/hvrh-b6nb.json')

if response.status_code == 200:
    try:
        data = response.json()
    except ValueError:
        print("Not JSON.")
else:
    print("Retrieve failed.")

In [3]:
df = pd.DataFrame.from_records(data)

In [4]:
df.to_csv("green_2016.csv", index=False)

In [5]:
connection_string = "DefaultEndpointsProtocol=https;AccountName=triathlonnyc;AccountKey=wfxBgbmGNXsXavVUF48Zn7sBj5ZNgT60MTArVyDKwn0CdqVebvaYyMWDzKfddso2yX0iiyr2fcK4+AStTsLANQ==;EndpointSuffix=core.windows.net"

In [6]:
container_name = "green"

In [7]:
blob_name = "green_2016"

In [8]:
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

In [9]:
container_client = blob_service_client.get_container_client(container_name)

In [11]:
# with open("green_2016.csv", "rb") as data:
#     container_client.upload_blob(name = blob_name, data=data)

In [21]:
# The base URL for the NYC Open Data portal's SODA API
base_url = "https://data.cityofnewyork.us/resource/"

# The discovery API endpoint for datasets
discovery_endpoint = "https://api.us.socrata.com/api/catalog/v1"

# Parameters for querying the dataset
params = {
    'domains': 'data.cityofnewyork.us',
    'search_context': 'data.cityofnewyork.us',
    'q': 'yellow taxi 2019'
}

# Send a GET request to the discovery endpoint with the parameters
response = requests.get(discovery_endpoint, params=params)

# Check if the request was successful
if response.status_code == 200:
    datasets = response.json()
    
    # Extract dataset identifiers and construct API endpoints
    for dataset in datasets['results']:
        resource = dataset['resource']
        dataset_id = resource['id']
        dataset_name = resource['name']
        api_endpoint = f"{base_url}{dataset_id}.json"
        
        print(f"Dataset Name: {dataset_name}")
        print(f"API Endpoint: {api_endpoint}")
else:
    print(f"Failed to retrieve datasets, status code: {response.status_code}")

Dataset Name: 2019 Yellow Taxi Trip Data
API Endpoint: https://data.cityofnewyork.us/resource/2upf-qytp.json
Dataset Name: Medallion Taxi Initial Inspection Schedule
API Endpoint: https://data.cityofnewyork.us/resource/sp7n-275u.json


In [34]:
def DatasetRetrieval2(year, colour):
    base_url = "https://data.cityofnewyork.us/resource/"

    # The discovery API endpoint for datasets
    discovery_endpoint = "https://api.us.socrata.com/api/catalog/v1"

    # Parameters for querying the dataset
    params = {
        'domains': 'data.cityofnewyork.us',
        'search_context': 'data.cityofnewyork.us',
        'q': f'{colour} taxi {year}'
    }

    # Send a GET request to the discovery endpoint with the parameters
    response = requests.get(discovery_endpoint, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        datasets = response.json()
        
        # Extract dataset identifiers and construct API endpoints
        for dataset in datasets['results']:
            resource = dataset['resource']
            dataset_id = resource['id']
            dataset_name = resource['name']
            api_endpoint = f"{base_url}{dataset_id}.json"
            
            if set(year).issubset(set(dataset_name)) and set(colour.lower()).issubset(set(dataset_name.lower())):
                print(dataset_name)
                return api_endpoint
    else:
        print(f"Failed to retrieve datasets, status code: {response.status_code}")

In [57]:
test_df = pd.read_parquet(f"https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2016-01.parquet")
test_df



,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2016-01-01 00:17:40,2016-01-01 00:36:15,N,1,255,228,1,8.46,25.5,0.5,0.5,30.00,0.0,None,0.3,56.80,1,1.0,NaN
1,2,2016-01-01 00:47:37,2016-01-01 01:18:37,N,1,106,36,1,5.84,23.0,0.5,0.5,4.86,0.0,None,0.3,29.16,1,1.0,NaN
2,2,2016-01-01 00:15:19,2016-01-01 00:24:27,N,2,226,260,1,1.55,52.0,0.0,0.5,0.00,0.0,None,0.3,52.80,2,1.0,NaN
3,2,2016-01-01 00:40:30,2016-01-01 00:45:42,N,1,226,260,1,0.88,5.5,0.5,0.5,0.00,0.0,None,0.3,6.80,2,1.0,NaN
4,2,2016-01-01 00:53:43,2016-01-01 01:43:47,N,1,226,79,2,9.06,37.0,0.5,0.5,0.00,0.0,None,0.3,38.30,2,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445287,2,2016-01-31 23:57:51,2016-02-01 00:05:15,N,1,41,24,1,1.38,8.0,0.5,0.5,0.00,0.0,None,0.3,9.30,2,1.0,NaN
1445288,2,2016-01-31 23:27:42,2016-01-31 23:32:31,N,1,223,223,1,1.11,6.0,0.5,0.5,1.82,0.0,None,0.3,9.12,1,1.0,NaN
1445289,2,2016-01-31 23:44:08,2016-01-31 23:50:35,N,1,223,223,1,1.43,7.0,0.5,0.5,0.00,0.0,None,0.3,8.30,2,1.0,NaN
1445290,2,2016-01-31 23:12:49,2016-01-31 23:24:15,N,1,61,49,1,2.34,10.5,0.5,0.5,0.00,0.0,None,0.3,11.80,2,1.0,NaN


In [75]:
def parquet_get(year, data_type, month):
    # months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    # df_list = []
    # for month in months:
    df_list = []
    url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{data_type}_tripdata_{year}-{month}.parquet"
    df = pd.read_parquet(url)
    df_list.append(df)
    return df_list

In [78]:
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
for month in months:
    a = parquet_get('2019', 'green', month)

print(a)

for df in a:
    df.to_csv(df_csv.csv)

[        VendorID lpep_pickup_datetime lpep_dropoff_datetime  \
0              1  2019-12-01 00:09:45   2019-12-01 00:10:59   
1              2  2019-12-01 00:26:05   2019-12-01 00:31:30   
2              2  2019-12-01 00:56:36   2019-12-01 00:59:38   
3              2  2019-12-01 00:26:20   2019-12-01 00:40:19   
4              2  2019-12-01 00:56:36   2019-12-01 00:59:56   
...          ...                  ...                   ...   
455289         2  2019-12-31 23:24:00   2019-12-31 23:40:00   
455290         2  2019-12-31 23:16:00   2019-12-31 23:37:00   
455291         2  2019-12-31 23:16:00   2019-12-31 23:37:00   
455292         2  2019-12-31 23:52:00   2020-01-01 00:05:00   
455293         2  2019-12-31 23:52:00   2020-01-01 00:05:00   

       store_and_fwd_flag  RatecodeID  PULocationID  DOLocationID  \
0                       N         1.0           145           145   
1                       N         1.0            24            41   
2                       N         1